# Kedro Project Explorer

This notebook helps you explore the Kedro project structure, source code, and data files.

## 1. Setup and Import

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd

# Add project source to path
project_path = Path.cwd().parent
sys.path.append(str(project_path / 'src'))

print(f"Project path: {project_path}")

Project path: /home/user/projects/kedro_project/account-tax


## 2. Explore Project Structure

In [ ]:
# List main directories
main_dirs = ['src', 'data', 'conf', 'notebooks', 'tests']

print("Project Structure:")
print("="*50)
for dir_name in main_dirs:
    dir_path = project_path / dir_name
    if dir_path.exists():
        print(f"\n📁 {dir_name}/")
        # List first level of contents
        for item in sorted(dir_path.iterdir())[:10]:  # Limit to first 10 items
            if item.is_dir():
                print(f"   📂 {item.name}/")
            else:
                print(f"   📄 {item.name}")

## 3. Explore Source Code

In [ ]:
# Explore src directory structure
src_path = project_path / 'src'

def explore_directory(path, prefix="", max_depth=3, current_depth=0):
    """Recursively explore directory structure"""
    if current_depth >= max_depth:
        return
    
    items = sorted(path.iterdir())
    for item in items:
        if item.name.startswith('.'):
            continue
        if item.is_dir() and item.name != '__pycache__':
            print(f"{prefix}📂 {item.name}/")
            explore_directory(item, prefix + "  ", max_depth, current_depth + 1)
        elif item.suffix == '.py':
            print(f"{prefix}📄 {item.name}")

print("Source Code Structure:")
print("="*50)
explore_directory(src_path)

In [ ]:
# Count Python files and lines of code
py_files = list(src_path.rglob('*.py'))
total_lines = 0

for py_file in py_files:
    with open(py_file, 'r', encoding='utf-8', errors='ignore') as f:
        total_lines += len(f.readlines())

print(f"Total Python files: {len(py_files)}")
print(f"Total lines of code: {total_lines:,}")

## 4. Explore Data Directory

In [ ]:
# Explore data directory
data_path = project_path / 'data'

print("Data Directory Structure:")
print("="*50)

data_layers = ['01_raw', '02_intermediate', '03_primary', '04_feature', 
               '05_model_input', '06_models', '07_model_output', '08_reporting']

for layer in data_layers:
    layer_path = data_path / layer
    if layer_path.exists():
        files = list(layer_path.glob('*'))
        print(f"\n📁 {layer}/ ({len(files)} files)")
        for file in sorted(files)[:5]:  # Show first 5 files
            if file.is_file():
                size = file.stat().st_size / 1024  # Size in KB
                print(f"   📄 {file.name} ({size:.1f} KB)")

In [ ]:
# List CSV and Parquet files in data directory
csv_files = list(data_path.rglob('*.csv'))
parquet_files = list(data_path.rglob('*.parquet'))

print(f"CSV files found: {len(csv_files)}")
print(f"Parquet files found: {len(parquet_files)}")

if csv_files:
    print("\nCSV files:")
    for csv_file in csv_files[:5]:  # Show first 5
        relative_path = csv_file.relative_to(data_path)
        print(f"  - {relative_path}")

if parquet_files:
    print("\nParquet files:")
    for parquet_file in parquet_files[:5]:  # Show first 5
        relative_path = parquet_file.relative_to(data_path)
        print(f"  - {relative_path}")

## 5. Load and Preview Data Files

In [ ]:
# Function to safely load and preview data
def preview_data_file(file_path, n_rows=5):
    """Load and preview a data file"""
    try:
        if file_path.suffix == '.csv':
            df = pd.read_csv(file_path, nrows=n_rows)
        elif file_path.suffix == '.parquet':
            df = pd.read_parquet(file_path).head(n_rows)
        else:
            print(f"Unsupported file type: {file_path.suffix}")
            return None
        
        print(f"\nFile: {file_path.name}")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        return df
    except Exception as e:
        print(f"Error loading {file_path.name}: {e}")
        return None

# Try to preview a data file
if csv_files:
    df = preview_data_file(csv_files[0])
    if df is not None:
        display(df.head())
elif parquet_files:
    df = preview_data_file(parquet_files[0])
    if df is not None:
        display(df.head())

## 6. Kedro Context and Catalog

In [ ]:
# Load Kedro context
try:
    from kedro.framework.session import KedroSession
    from kedro.framework.startup import bootstrap_project
    
    # Bootstrap the project
    bootstrap_project(project_path)
    
    # Create a session
    with KedroSession.create(project_path=project_path) as session:
        context = session.load_context()
        catalog = context.catalog
        
        # List available datasets
        datasets = list(catalog.list())
        print(f"Available datasets in catalog: {len(datasets)}")
        print("\nFirst 10 datasets:")
        for dataset_name in sorted(datasets)[:10]:
            print(f"  - {dataset_name}")
            
except Exception as e:
    print(f"Could not load Kedro context: {e}")
    print("Make sure you're running from the project directory")

## 7. Pipeline Information

In [ ]:
# List available pipelines
try:
    from kedro.framework.session import KedroSession
    
    with KedroSession.create(project_path=project_path) as session:
        context = session.load_context()
        pipelines = context.pipelines
        
        print(f"Available pipelines: {list(pipelines.keys())}")
        
        # Show pipeline nodes
        for pipeline_name, pipeline in pipelines.items():
            print(f"\n{pipeline_name} pipeline:")
            print(f"  Nodes: {len(pipeline.nodes)}")
            if pipeline.nodes:
                print("  First 5 nodes:")
                for node in pipeline.nodes[:5]:
                    print(f"    - {node.name}")
                    
except Exception as e:
    print(f"Could not load pipelines: {e}")

## 8. Configuration Files

In [ ]:
# Explore configuration files
conf_path = project_path / 'conf' / 'base'

print("Configuration files:")
print("="*50)

yaml_files = list(conf_path.glob('*.yml')) + list(conf_path.glob('*.yaml'))
for yaml_file in sorted(yaml_files):
    size = yaml_file.stat().st_size / 1024  # Size in KB
    print(f"📄 {yaml_file.name} ({size:.1f} KB)")

# Read parameters if available
params_file = conf_path / 'parameters.yml'
if params_file.exists():
    import yaml
    with open(params_file, 'r') as f:
        params = yaml.safe_load(f)
    print(f"\nParameters keys: {list(params.keys()) if params else 'None'}")

## 9. Quick Data Analysis Helper

In [ ]:
# Helper function to analyze any data file
def analyze_data_file(file_path):
    """Provide quick analysis of a data file"""
    try:
        print(f"Analyzing: {file_path}")
        print("="*50)
        
        # Load data
        if file_path.suffix == '.csv':
            df = pd.read_csv(file_path)
        elif file_path.suffix == '.parquet':
            df = pd.read_parquet(file_path)
        else:
            print(f"Unsupported file type: {file_path.suffix}")
            return
        
        # Basic info
        print(f"Shape: {df.shape}")
        print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        print(f"\nColumn types:")
        print(df.dtypes.value_counts())
        print(f"\nMissing values:")
        missing = df.isnull().sum()
        if missing.any():
            print(missing[missing > 0])
        else:
            print("No missing values")
        
        # Show sample
        print(f"\nFirst 3 rows:")
        display(df.head(3))
        
        # Numeric columns statistics
        numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
        if len(numeric_cols) > 0:
            print(f"\nNumeric columns statistics:")
            display(df[numeric_cols].describe())
            
    except Exception as e:
        print(f"Error analyzing file: {e}")

# Example usage (uncomment and modify path):
# analyze_data_file(project_path / 'data' / '01_raw' / 'your_file.csv')

## 10. Your Work Space

Use the cells below for your own exploration:

In [1]:
from load_intermediate_outputs import (
    get_catalog,
    load_prepared_datasets,
    load_text_datasets,
    summarise_dataframe,
)

[09/25/25 13:19:23] INFO     Using                                                                  ]8;id=326210;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=913909;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/                
                             kedro/framework/project/rich_logging.yml' as logging configuration.                   

In [2]:
prepared = load_prepared_datasets()  # returns {"train": DataFrame, "valid": ..., "test": ...}
textual = load_text_datasets()

[09/25/25 13:19:50] WARNING  /home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/m ]8;id=779944;file:///usr/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=263962;file:///usr/lib/python3.12/warnings.py#112\112]8;;\
                             lflow/pyfunc/utils/data_validation.py:186: UserWarning: [33mAdd type                  
                             hints to the `predict` method to enable data validation and automatic                 
                             signature inference during model logging. Check                                       
                             https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage                
                             -in-pythonmodel for more details.[0m                                                  
                               color_warning(                                                                      
                                                                                                                   

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=878254;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=377946;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=113169;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=298689;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#293\293]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///home/user/projects/kedro_project/account-tax/mlrun                           
                             s'                                                                                    

                    WARNING  /home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/t ]8;id=117688;file:///usr/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=551659;file:///usr/lib/python3.12/warnings.py#112\112]8;;\
                             qdm/auto.py:21: TqdmWarning: IProgress not found. Please update                       
                             jupyter and ipywidgets. See                                                           
                             https://ipywidgets.readthedocs.io/en/stable/user_install.html                         
                               from .autonotebook import tqdm as notebook_tqdm                                     
                                                                                                                   

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


...

[09/25/25 13:19:52] INFO     Loading data from prepared_datasets_mlflow                        ]8;id=302602;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=419781;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\
                             (MlflowPartitionedParquetDataset)...                                                  

[09/25/25 13:19:53] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=419127;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=498965;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=550129;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=298035;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#293\293]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///home/user/projects/kedro_project/account-tax/mlrun                           
                             s'                                                                                    

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=949453;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=630267;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/25/25 13:19:54] INFO     Loading data from text_datasets_mlflow                            ]8;id=125200;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=982675;file:///home/user/projects/kedro_project/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\
                             (MlflowPartitionedParquetDataset)...                                                  

In [3]:
prepared["train"].head()

,date,vat_type,party_name,company_name,company_vat_id,item,supply_amount,vat_amount,total_amount,in_multi,...,business_item,vat_message_code,close_date,product,company_business_type,company_business_item,company_vat_message_code,acct_code,labels,split
0,1970-01-01 00:00:00.020240131,51,지엠아이(GMI),주식회사 코윈,4438102301,보호테이프 외,5121180,512118.0,5633298.0,0.0,...,전자상거래 소매업,standard,None,"자동차부품,점착테이프",제조업,그 외 자동차용 신품 부품 제조업,standard,15300,1,train
1,1970-01-01 00:00:00.020230411,57,주식회사 씨엔씨식자재유통,소울푸드 주식회사,1878702515,None,21891,2189.0,24080.0,NaN,...,슈퍼마켓,standard,None,식잡,도매 및 소매업,기타 음ㆍ식료품 위주 종합 소매업,standard,14600,10,train
2,1970-01-01 00:00:00.020191004,57,한밭골 사골 순대＆정다운 다,유니슨방음방진(주),3188107301,한밭골 사골 순대＆정다운 다락방,7273,727.0,8000.0,0.0,...,None,standard,20220428,None,제조업,산업용 그 외 비경화 고무제품 제조업,standard,81100,36,train
3,1970-01-01 00:00:00.020230731,51,네이버파이낸셜 주식회사,(주)팔도로지스,2338100176,네이버페이 수수료,284160,28416.0,312576.0,NaN,...,그 외 기타 금융 지원 서비스업,standard,None,온라인 정보 제공/전자지급결제대행,도매 및 소매업,"빵류, 과자류, 당류, 초콜릿 도매업",standard,83100,24,train
4,1970-01-01 00:00:00.020241228,51,용달화물,주식회사 큐앤비푸드,2468801205,운송료,250000,25000.0,275000.0,0.0,...,None,standard,None,None,제조업,과자류 및 코코아 제품 제조업,standard,52400,64,train


In [4]:
prepared.decribe

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prepared.decribe                                                                             │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'dict' object has no attribute 'decribe'

In [5]:
prepared.describe

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prepared.describe                                                                            │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'dict' object has no attribute 'describe'

In [6]:
type(prepared)

<class 'dict'>

In [7]:
prepared


{
    'test':                                 date  vat_type   party_name     company_name  \
0      1970-01-01 00:00:00.020231030        57         종합볼트          (주)퀀텀테크   
1      1970-01-01 00:00:00.020230511        57       서울명동찌개  주식회사(YG기초)와이지기초   
2      1970-01-01 00:00:00.020230225        54      롯데렌탈(주)        메이저씨엔엠(주)   
3      1970-01-01 00:00:00.020231016        70       우정사업본부        주식회사 꿈의날개   
4      1970-01-01 00:00:00.020220601        51       유병기부동산       주식회사 엔알시스템   
...                              ...       ...          ...              ...   
330288 1970-01-01 00:00:00.020230430        11  효산의료재단 지샘병원          주식회사 영광   
330289 1970-01-01 00:00:00.020220321        51         개인용달      화물나라종합물류(주)   
330290 1970-01-01 00:00:00.020210222        11       부뚜막손두부          주식회사 장원   
330291 1970-01-01 00:00:00.020210107        51        대동천막사       (주)한성엔지니어링   
330292 1970-01-01 00:00:00.020240807        51     주식회사 케이티        (주)유진콤프레샤   

       company_vat_id   

In [8]:
prepared.shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prepared.shape                                                                               │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'dict' object has no attribute 'shape'

In [9]:
prepared.keys()

dict_keys(['test', 'train', 'valid'])

In [10]:
prepared.head()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prepared.head()                                                                              │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'dict' object has no attribute 'head'

In [11]:
def print_dict_structure(mapping, indent=0):
    prefix = " " * indent
    if isinstance(mapping, dict):
        for key, value in mapping.items():
            print(f"{prefix}- {key}: {type(value).__name__}")
            print_dict_structure(value, indent + 2)
    else:
        return  # dict가 아닌 값에 도달하면 종료

print_dict_structure(prepared)


- test: DataFrame
- train: DataFrame
- valid: DataFrame


In [12]:
prepared["train"].describe(include="all")

,date,vat_type,party_name,company_name,company_vat_id,item,supply_amount,vat_amount,total_amount,in_multi,...,business_item,vat_message_code,close_date,product,company_business_type,company_business_item,company_vat_message_code,acct_code,labels,split
count,2642343,2.642343e+06,2641545,2642343,2642343,1204781,2.642343e+06,2.583423e+06,2.642342e+06,842277.000000,...,2028263,2642343,317276,1124071,2554758,2576820,2642343,2.642343e+06,2.642343e+06,2642343
unique,NaN,NaN,226732,380,371,306663,NaN,NaN,NaN,NaN,...,1126,8,2318,12519,14,170,3,NaN,NaN,1
top,NaN,NaN,비씨카드 (주),주식회사 장원,3128132710,유지보수비,NaN,NaN,NaN,NaN,...,한식 일반 음식점업,standard,20241231,보험서비스/신용카드,제조업,택시 운송업,standard,NaN,NaN,train
freq,NaN,NaN,64945,73945,73945,65921,NaN,NaN,NaN,NaN,...,174225,2464637,9054,76242,1271832,140703,2620520,NaN,NaN,2642343
mean,1970-01-01 00:00:00.020220029,5.038250e+01,NaN,NaN,NaN,NaN,2.535555e+06,2.282584e+05,2.758724e+06,0.108092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.355957e+04,3.927025e+01,NaN
min,1970-01-01 00:00:00.020190101,1.100000e+01,NaN,NaN,NaN,NaN,-8.830405e+12,-1.000000e+10,-8.830405e+12,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.010000e+04,0.000000e+00,NaN
25%,1970-01-01 00:00:00.020210204,5.100000e+01,NaN,NaN,NaN,NaN,1.118100e+04,1.090000e+03,1.225500e+04,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.230000e+04,1.300000e+01,NaN
50%,1970-01-01 00:00:00.020220901,5.700000e+01,NaN,NaN,NaN,NaN,4.545500e+04,4.545000e+03,5.000000e+04,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.110000e+04,2.800000e+01,NaN
75%,1970-01-01 00:00:00.020231130,5.700000e+01,NaN,NaN,NaN,NaN,2.561275e+05,2.320000e+04,2.805000e+05,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.220000e+04,6.200000e+01,NaN
max,1970-01-01 00:00:00.020250330,7.000000e+01,NaN,NaN,NaN,NaN,8.830405e+12,1.000000e+10,8.830405e+12,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.800000e+04,2.590000e+02,NaN
